In [1]:
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib.pyplot as plt
import re
import seaborn as sns
sns.set(color_codes=True)

import tabula
from tabula.io import read_pdf

In [2]:
df_pdf = read_pdf('kcp-academ.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

df = df_pdf[0]
for df_temp in df_pdf[1:]:
    df = df.append(df_temp,ignore_index=True)

In [3]:
df

,0,1,2,3,4
0,Код и наименование\rнаправления подготовки,Направленность (профиль/специализация) образов...,Контрольные цифры\rприема (бюджет),Цифры приема по договорам\rоб оказании платных...,Цифры приема по договорам об\rоказании платных...
1,БАКАЛАВРИАТ,NaN,NaN,NaN,NaN
2,Очная форма обучения,NaN,NaN,NaN,NaN
3,01.03.02\rПрикладная математика и\rинформатика,Прикладные цифровые технологии\r(Школа ИТ- мен...,0,35,10
4,09.03.03\rПрикладная информатика,Прикладная информатика в экономике (ФИТАД ЭМИТ...,15,55,5
...,...,...,...,...,...
126,38.05.01\rЭкономическая безопасность,Экономико-правовое обеспечение экономической б...,18,65,5
127,38.05.02\rТаможенное дело,Организация таможенного контроля (ФТД ИПНБ)\rТ...,20,75,5
128,40.05.01\rПравовое обеспечение\rнациональной б...,Гражданско-правовая (ЮФ ИПНБ)\rГосударственно-...,5,150,5
129,40.05.04\rСудебная и прокурорская\rдеятельность,Судебная деятельность (ЮФ ИПНБ)\rПрокурорская ...,3,80,5


In [4]:
df_one = df.iloc[3:,:2][ ~df[0].str.contains('Код|СПЕЦИАЛИТЕТ|Очно-заочная форма обучения|Заочная форма обучения|Очная форма обучения',
                                            na=False) ].reset_index(drop = True)
df_one

C:\Users\arakh\AppData\Local\Temp/ipykernel_85084/2927323178.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_one = df.iloc[3:,:2][ ~df[0].str.contains('Код|СПЕЦИАЛИТЕТ|Очно-заочная форма обучения|Заочная форма обучения|Очная форма обучения',


,0,1
0,01.03.02\rПрикладная математика и\rинформатика,Прикладные цифровые технологии\r(Школа ИТ- мен...
1,09.03.03\rПрикладная информатика,Прикладная информатика в экономике (ФИТАД ЭМИТ...
2,09.03.03\rПрикладная информатика,Прикладная информатика в цифровой экономике\r(...
3,09.03.03\rПрикладная информатика,Системы искусственного интеллекта (прикладная ...
4,09.03.03\rПрикладная информатика,Анализ данных (ФИТАД ЭМИТ)
...,...,...
110,38.05.01\rЭкономическая безопасность,Экономико-правовое обеспечение экономической б...
111,38.05.02\rТаможенное дело,Организация таможенного контроля (ФТД ИПНБ)\rТ...
112,40.05.01\rПравовое обеспечение\rнациональной б...,Гражданско-правовая (ЮФ ИПНБ)\rГосударственно-...
113,40.05.04\rСудебная и прокурорская\rдеятельность,Судебная деятельность (ЮФ ИПНБ)\rПрокурорская ...


In [5]:
def program_name(patern, tmp):
    tmp = tmp.replace('\r'," ").split(':',1)[1]
    #tmp = tmp.replace('  '," ")
    tmp = tmp.replace(re.findall(patern, tmp )[-1],'')
    " ".join(tmp.split())
    tmp = tmp.strip()
    return tmp

def facult_name(patern, tmp):
    tmp = tmp.replace('\r'," ")
    tmp = re.findall(patern, tmp )[-1].strip()
    return tmp

df_one_mos = pd.DataFrame([])
df_one_mos['код'] = df_one[0].map(lambda x: x.split('\r',1)[0])
df_one_mos['стандарт'] = df_one[0].map(lambda x: ' '.join(''.join(x.split('\r',1)[1:3]).split('\r',1)))
#df_one_mos['программа'] = df_one[1].map(lambda x: x.split('\r',4)[0]).map(lambda x: x.split('(',4)[0])
#df_one_mos['программа'] =  df_one.iloc[:,1].map(lambda x: program_name("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x))
df_one_mos['подразделение'] =  df_one.iloc[:,1].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
#df_one_mos['программа'] = df_one_mos['программа'].map(lambda x: re.sub('[ё]', 'е', x))
pd.set_option('display.max_rows', None)
display(df_one_mos)

,код,стандарт,подразделение
0,01.03.02,Прикладная математика и информатика,Школа ИТ- менеджмент ЭМИТ
1,09.03.03,Прикладная информатика,ФИТАД ЭМИТ
2,09.03.03,Прикладная информатика,ФИТАД ЭМИТ
3,09.03.03,Прикладная информатика,ФИТАД ЭМИТ
4,09.03.03,Прикладная информатика,ФИТАД ЭМИТ
5,09.03.03,Прикладная информатика,ФИТАД ЭМИТ
6,27.03.02,Управление качеством,ФММС ИУРР
7,27.03.05,Инноватика,ИОМ
8,37.03.01,Психология,ИОН
9,37.03.01,Психология,ИОН


In [6]:
df_one.iloc[:,1].map(lambda x: program_name("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x))

IndexError: list index out of range

In [ ]:
df_one.iloc[:,1]

In [ ]:
df_one[1].map(lambda x: x.split('\r',4)[0]).map(lambda x: x.split('(',4)[1])
#.__contains__('(')

In [ ]:
df.iloc[3:,1][15]
#.map(lambda x: x.split('\r',1))

In [ ]:
df.iloc[3:,1].map(lambda x: x.split('\r',1))

In [ ]:
df.iloc[3:,1][20].split('\r',1)

In [ ]:
df.iloc[:,1][20].split('\r',1)

In [ ]:
df[1].split('\r',1)

In [ ]:
df.iloc[:,1].map(lambda x: x.split('\r',1)[1])

In [ ]:
df.iloc[3:,0][3].split('\r',1)[1].contains('\r', na=False)

In [ ]:
df.iloc[3:,0].map(lambda x: x.split('\r',2))
#dff[dff.str.contains('\r', na=False)]


In [ ]:
df[df[0].str.contains('\r', na=False)]

In [ ]:
df.iloc[3:,0].map(lambda x: ''.join(x[10:30]))

In [ ]:
' '.join(''.join(df.iloc[3:,0][3].split('\r',1)[1:3]).split('\r',1))


In [ ]:
df.iloc[3:,0][3].split('\r',1)

In [ ]:
sr = 'Прикладная математика и\rинформатика'
sr.__contains__('\r')

In [ ]:
df.iloc[3:,0][3].split('\r',1)[1:3].__contains__('\r')

In [ ]:
def get(i):
    return ' '.join(''.join(df.iloc[3:,0][i].split('\r',1)[1:3]).split('\r',1))
get(3)

In [ ]:
def gett(i):
    return ''.join(df.iloc[3:,0][i].split('\r',1)[1:3])
gett(3)

In [ ]:
df.iloc[3:,0].map(lambda x: ' '.join(''.join(x.split('\r',1)[1:3]).split('\r',1)))

In [7]:
df_one[1][4].split('\r',3)[1]

IndexError: list index out of range

In [8]:
' '.join(df_one[1][4].split('\r',1)).__contains__('(')

True

In [9]:
df_one[1].__contains__('(')

False

In [10]:
#df_one[1]
df_one[1].map(lambda x: x.split('(',1)[1]).map(lambda x: x.split(')',1)[0])

0                              Школа ИТ- менеджмент ЭМИТ
1                                             ФИТАД ЭМИТ
2                                             ФИТАД ЭМИТ
3                                 прикладная информатика
4                                             ФИТАД ЭМИТ
5                                             ФИТАД ЭМИТ
6                                              ФММС ИУРР
7                                                    ИОМ
8                                                    ИОН
9                                                    ИОН
10                                               ЭФ ЭМИТ
11                                               ЭФ ЭМИТ
12                                              ЭФ\rЭМИТ
13                                                   ФФБ
14                                             ФММС ИУРР
15                                                  ИГСУ
16                       российско-французская программа
17                             

In [11]:
#' '.join(' '.join(
df_one[1].map(lambda x: x.split('(',1)[1]).map(lambda x: x.split(')',1)[0]).map(lambda x: x.split('\r',2)[0])
#.map(lambda x: x.split(')',2)[1])

0                              Школа ИТ- менеджмент ЭМИТ
1                                             ФИТАД ЭМИТ
2                                             ФИТАД ЭМИТ
3                                 прикладная информатика
4                                             ФИТАД ЭМИТ
5                                             ФИТАД ЭМИТ
6                                              ФММС ИУРР
7                                                    ИОМ
8                                                    ИОН
9                                                    ИОН
10                                               ЭФ ЭМИТ
11                                               ЭФ ЭМИТ
12                                                    ЭФ
13                                                   ФФБ
14                                             ФММС ИУРР
15                                                  ИГСУ
16                       российско-французская программа
17                             

In [12]:
def multi_transform(tmp):
    tmp_3 = tmp.iloc[1].split(':\r',1)[1]
    tmp_3 = " ".join(tmp_3.split())
    tmp_3 = re.split('\d.',tmp_3)[1:]
    tmp_3 = np.char.replace(tmp_3,'\r',' ')
    tmp_3 = np.char.replace(tmp_3,'.','')
    dic = {1: tmp.iloc[0],
           2: tmp.iloc[1].split('\r',1)[0],
           3: tmp_3
          }
    df_tmp = pd.DataFrame(dic)
    df_tmp[4] = df_tmp[3].map(lambda x: facult_name("\(([ а-яa-zА-ЯA-Z0-9-: ]*)\)", x))
    df_tmp[3] = df_tmp[3].map(lambda x: x.replace(re.findall("\([ а-яa-zА-ЯA-Z0-9-: ]*\)", x )[-1],'').strip() )
    return df_tmp
multi_transform(df_one[1])

IndexError: list index out of range

In [13]:
df_one[1].map(lambda x: x.split('(',1)[1]).map(lambda x: x.split(')',1)[0])[4].isupper()

True

In [14]:
df_one[1][4].strip('(')
#.isupper()

'Анализ данных (ФИТАД ЭМИТ)'

In [15]:
#'[ert]'.replace('[', 'u'
w = 'retyr]'.strip(']')
w

'retyr'

In [16]:
df_one[1][4].replace('\r', ' ')

'Анализ данных (ФИТАД ЭМИТ)'

In [17]:
df_one[1][9].split('(',1)[0].strip()

'Психологическое консультирование и коучинг'

In [18]:
df_one[1][9].replace('\r'," ").split(')',3)[2].split('(',1)[0]

IndexError: list index out of range

In [19]:
def por(str, i):
    if ((i != 0) & (i != 5)):
        for j in range(0,i):
            print(str.replace('\r'," ").split(')',i)[j].split('(',1)[0].strip())
    else:
        print(str.split('(',1)[0].strip())
por(df_one[1][9], df_one[1][9].count("\r"))

Психологическое консультирование и коучинг


In [20]:
df_one[1].map(lambda x: por(x, x.count("\r")))

Прикладные цифровые технологии
Прикладная информатика в экономике
Цифровые технологии
Прикладная информатика в информационной безопасности
Прикладная информатика в цифровой экономике
Системы искусственного интеллекта
Анализ данных
Веб-разработка
системахие качеством в производственно-технологических
Технологическое предпринимательство
Психология: современная теория и практика
Психологическое консультирование и коучинг
Экономика и финансы
Прикладная математика и экономика
Мировая экономика и внешнеэкономическая деятельность
Финансы и кредит
Учет, анализ и аудит
Деньги, банки, финансовые рынки
Финансы и ESG – трансформация бизнеса
Финансовые рынки и цифровые технологии
Предпринимательство и управление эффективностью бизнеса
Финансы государственных корпораций и цифровые сервисы

Торговая политика
Управление рисками и страховая деятельность
Экономика в топливно-энергетическом комплексе
Финансовый контроль и государственный аудит
Финансовый контроль и государственный аудит
Финансы и инвести

IndexError: list index out of range

In [21]:
df_one[1]

0      Прикладные цифровые технологии\r(Школа ИТ- мен...
1      Прикладная информатика в экономике (ФИТАД ЭМИТ...
2      Прикладная информатика в цифровой экономике\r(...
3      Системы искусственного интеллекта (прикладная ...
4                             Анализ данных (ФИТАД ЭМИТ)
5                            Веб-разработка (ФИТАД ЭМИТ)
6      Управление качеством в производственно-техноло...
7      Технологическое предпринимательство (ИОМ)\rМеж...
8        Психология: современная теория и практика (ИОН)
9       Психологическое консультирование и коучинг (ИОН)
10     Экономика и финансы (ЭФ ЭМИТ)\rЦифровая эконом...
11           Прикладная математика и экономика (ЭФ ЭМИТ)
12     Мировая экономика и внешнеэкономическая деятел...
13     Финансы и кредит (ФФБ)\rУчет, анализ и аудит (...
14     Торговая политика (ФММС ИУРР)\rУправление риск...
15     Финансовый контроль и государственный аудит (И...
16     Финансовый контроль и государственный аудит\r(...
17                           Фи

In [22]:
def program_name(patern, tmp):
    tmp = tmp.replace('\r'," ").split(')',1)[1]
    #tmp = tmp.replace('  '," ")
    tmp = tmp.replace(re.findall(patern, tmp )[-1],'')
    " ".join(tmp.split())
    tmp = tmp.strip()
    return tmp

In [23]:
df_pdff = read_pdf('PP-2022-pril-2_fil.pdf', 
    pages= 'all',
    lattice=True,
    pandas_options={"header": [0, 1, 2, 3]},
    #area=[0, 0, 50, 100],
    relative_area=True,
    #multiple_tables=False
                        )

dff = df_pdff[0]
for df_temp in df_pdff[1:]:
    dff = dff.append(df_temp,ignore_index=True)

Got stderr: июн 05, 2022 11:41:21 AM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



In [42]:
dff.head(10)
fil = dff.iloc[:,0:3][~dff[0].str.contains("Код|Очная форма обучения|Очно-заочная форма обучения|Заочная форма обучения", na=False)]

In [25]:
fil[0].dropna()

2                                                       1
3                                        АЛТАЙСКИЙ ФИЛИАЛ
5                                    37.03.01\rПсихология
8                                    37.03.01\rПсихология
11                                    38.03.01\rЭкономика
14                                    38.03.01\rЭкономика
17      38.03.04\rГосударственное и\rмуниципальное упр...
20      38.03.04\rГосударственное и\rмуниципальное упр...
23                                40.03.01\rЮриспруденция
26                                40.03.01\rЮриспруденция
30                            42.03.05\rМедиакоммуникации
31                                        3. Русский язык
32                       Предмет по выбору\rпоступающего:
33                                      2. Обществознание
34                                             2. История
37                                                      1
38                                   37.03.01\rПсихология
41            

In [26]:

#fil.reset_index(drop = True)

In [27]:
fil.iloc[:,3]

IndexError: single positional indexer is out-of-bounds

In [ ]:
filial = pd.DataFrame()
filial['код'] = fill.iloc[:,1].map(lambda x: x.replace('\r', ' ')).map(lambda x: x.split(' ', 1)[0])
filial['стандарт'] = fill.iloc[:,1].map(lambda x: x.replace('\r', ' ')).map(lambda x: x.split(' ', 1)[1])
#filial['программа'] = fill.iloc[:,3].map(lambda x:get_pr(x))
filial.head(10)

In [ ]:
def get_pr(strr):
    if strr.split('\r', 1)[0].__contains__('Образовательная') == True:
        print(' '.join(''.join(strr.split('\r',1)[1]).split('\r',1)))
    else:
        a = strr.count('\r')
        for i in range(1, a+1):
            print(' '.join(''.join(strr.split('\r',a)[i]).split('\r',1)).strip('1.,2.,3.').strip())
        
get_pr(fill.iloc[:,3][2])

In [ ]:
fil

In [ ]:
fil.iloc[:,3].map(lambda x:get_pr(x))

In [ ]:
fil.iloc[:,3].map(lambda x: x.split('\r', 1)[1])

In [ ]:
fil.iloc[:,1].map(lambda x: ' '.join(''.join(x.split('\r',1)[1:3]).split('\r',1)))

In [ ]:
len(fil[~fil[0].str.contains('ФИЛИАЛ|Русский', na = False)][0].dropna())

In [ ]:
fil[2].dropna()

In [ ]:
a = []
b = []
c = []
def fi(fil):
    for i in range(0,len(fil.iloc[:,3])):
        if (fil.iloc[:,3][i].split('\r', 1)[0].__contains__('Образовательная') == True):
            a.append(fil.iloc[:,1][i].split('\r')[0])
            b.append(fil.iloc[:,1][i].split('\r')[1])
            c.append((' '.join(''.join(fil.iloc[:,3][i].split('\r',1)[1]).split('\r',1))))
        else:
            aa = fil.iloc[:,3][i].count('\r')
            for j in range(1, aa+1):
                a.append(fil.iloc[:,1][i].split('\r')[0])
                b.append(fil.iloc[:,1][i].split('\r')[1])
                c.append((' '.join(''.join(fil.iloc[:,3][i].split('\r',aa)[j]).split('\r',1)).strip('1234567890.').strip()))
        
fi(fill)

In [ ]:
filiall = pd.DataFrame()
filiall['код'] = a
filiall['стандарт'] = b
filiall['программа'] = c
filiall.head(20)

In [ ]:
df_one.head()

In [ ]:
df_one[1][0]

In [ ]:
q = []
w = []
e = []
def mos():
    for i in range(0,len(df_one.iloc[:,1])):
        if (df_one[1][i].count('\r') <= 1):
            q.append(df_one.iloc[:,0][i].split('\r')[0])
            w.append(df_one.iloc[:,0][i].split('\r')[1])
            e.append((' '.join(''.join(df_one.iloc[:,1][i].split('\r',1)[1]).split('\r',1))))
        else:
            aa = df_one.iloc[:,1][i].count('\r')
            for j in range(1, aa+1):
                q.append(df_one.iloc[:,0][i].split('\r')[0])
                w.append(df_one.iloc[:,0][i].split('\r')[1])
                e.append((' '.join(''.join(df_one.iloc[:,1][i].split('\r',aa)[j]).split('\r',1)).strip()))
        
mos()

In [ ]:
#print(df_one.iloc[:,1][1])
df_one.iloc[:,1][1].split('\r',1)[0].split('(',1)[0].strip()
df_one.iloc[:,1][1].split('\r',1)[0].split('(',1)[1].replace(')','')
df_one.iloc[:,1][1].split('\r',1)[1].split('(',1)[0].strip()
df_one.iloc[:,1][1].split('\r',1)[1].split('(',1)[1].replace(')','')

In [ ]:
df_one.iloc[:,1][2].split('\r',1)[1].split('(',1)[1].strip(')')

In [ ]:
qq = []
def mos():
    for i in range(0,len(df_one.iloc[:,1])):
        if (df_one[1][i].count('\r') <= 1):
            qq.append(df_one.iloc[:,1][i].split('\r',1)[0].split('(',1)[0].strip())
            #w.append(df_one.iloc[:,1][i].split('\r',1)[1].split('(',1)[1].strip(')'))
            #e.append((' '.join(''.join(df_one.iloc[:,1][i].split('\r',1)[1]).split('\r',1))))
mos()

In [ ]:
a = []
b = []
c = []
d = []
def fi():
    for i in range(0,len(fil.iloc[:,2])):
            #ddf = copy(fil)
            #if (fil.iloc[:,1][i].__contains__('ФИЛИАЛ')):
            #    fili = fil.iloc[:,1][i]
            #    i = i + 1
            if (fil.iloc[:,3][i].split('\r', 1)[0].__contains__('Образовательная') == True):
                a.append(fil.iloc[:,1][i].split('\r')[0])
                b.append(fil.iloc[:,1][i].split('\r')[1])
                c.append((' '.join(''.join(fil.iloc[:,3][i].split('\r',1)[1]).split('\r',1))))
                d.append(fili)
            elif (fil.iloc[:,3][i].split('\r', 1)[0].__contains__('Совокупность') == True):
                aa = fil.iloc[:,3][i].count('\r')
                for j in range(1, aa+1):
                    a.append(fil.iloc[:,1][i].split('\r')[0])
                    b.append(fil.iloc[:,1][i].split('\r')[1])
                    c.append((' '.join(''.join(fil.iloc[:,3][i].split('\r',aa)[j]).split('\r',1)).strip('1234567890.').strip()))
                    d.append(fili)
fi()


In [38]:
st

0                                       АЛТАЙСКИЙ ФИЛИАЛ
1                                   37.03.01\rПсихология
2                                   37.03.01\rПсихология
3                                    38.03.01\rЭкономика
4                                    38.03.01\rЭкономика
5      38.03.04\rГосударственное и\rмуниципальное упр...
6      38.03.04\rГосударственное и\rмуниципальное упр...
7                                40.03.01\rЮриспруденция
8                                40.03.01\rЮриспруденция
9                            42.03.05\rМедиакоммуникации
10                                  37.03.01\rПсихология
11                                   38.03.01\rЭкономика
12     38.03.04\rГосударственное и\rмуниципальное упр...
13                               40.03.01\rЮриспруденция
14                               40.03.01\rЮриспруденция
15                            40.03.01\rЮриспруденция***
16                           42.03.05\rМедиакоммуникации
17       40.05.04\rСудебная и п

In [ ]:

#st

In [90]:
a = []
b = []
c = []
d = []
e = []

def fi(st):
    for i in range(0,len(fill.iloc[:,2])):
            if (st[i].__contains__('ФИЛИАЛ')):
                fili = st[i].capitalize()
                st = st.drop(i).reset_index(drop = True)
            if ((st[i].__contains__('БАКАЛАВРИАТ')) or (st[i].__contains__('СПЕЦИАЛИТЕТ'))):
                bak = st[i].capitalize()
                st = st.drop(i).reset_index(drop = True)
                
            if (fill.iloc[:,3][i].split('\r', 1)[0].__contains__('Образовательная') == True):
                a.append(fill.iloc[:,1][i].split('\r')[0])
                b.append(' '.join(st[i].split('\r')[1:(st[i]).count('\r')+1]).strip('*'))
                c.append((' '.join(''.join(fill.iloc[:,3][i].split('\r',1)[1]).split('\r',1))))
                d.append(fili)
                e.append(bak)
            elif (fill.iloc[:,3][i].split('\r', 1)[0].__contains__('Совокупность') == True):
                for j in range(1, fill.iloc[:,3][i].count('.')):
                    #c.append(fill[fill.iloc[:,3].str.contains('Совокупность')][2].reset_index(drop = True)[20].split('.')[j].replace('\r', ' ').strip('1234567890. '))
                    c.append(fill.iloc[:,3][i].split('.')[j].replace('\r', ' ').strip('1234567890. '))
                    a.append(fill.iloc[:,1][i].split('\r')[0])
                    b.append(' '.join(st[i].split('\r')[1:(st[i]).count('\r')+1]).strip('*'))
                    #c.append((' '.join(''.join(fill.iloc[:,3][i].split('\r',fill.iloc[:,3][i].count('\r'))[j]).split('\r',1)).strip('1234567890.').strip()))
                    d.append(fili)
                    e.append(bak)
fi(st)
filiall = pd.DataFrame()
filiall['код'] = a
filiall['стандарт'] = b
filiall['программа'] = c
filiall['филиал'] = d
filiall['уровень'] = e
filiall = filiall.drop_duplicates().reset_index(drop = True)
filiall

,код,стандарт,программа,филиал,уровень
0,37.03.01,Психология,Психология управления,Алтайский филиал,Бакалавриат
1,38.03.01,Экономика,Экономика и управление финансами,Алтайский филиал,Бакалавриат
2,38.03.04,Государственное и муниципальное управление,Административное управление,Алтайский филиал,Бакалавриат
3,40.03.01,Юриспруденция,Уголовно-правовой профиль,Алтайский филиал,Бакалавриат
4,40.03.01,Юриспруденция,Государственно-правовой профиль,Алтайский филиал,Бакалавриат
5,42.03.05,Медиакоммуникации,Медиакоммуникации и продвижение в цифровой среде,Алтайский филиал,Бакалавриат
6,40.05.04,Судебная и прокурорская деятельность,Прокурорская деятельность,Алтайский филиал,Специалитет
7,38.03.01,Экономика,Финансы и кредит,Астраханский филиал,Бакалавриат
8,38.03.02,Менеджмент,Менеджмент организации,Астраханский филиал,Бакалавриат
9,38.03.04,Государственное и муниципальное управление,Государственная и муниципальная служба,Астраханский филиал,Бакалавриат


In [ ]:
' '.join(fill.iloc[16,1].split('\r')[1:(fill.iloc[4,1]).count('\r')+1])

In [36]:
filiall

NameError: name 'filiall' is not defined

In [ ]:
' '.join(fill.iloc[6,1].split('\r')[1:(fill.iloc[4,1]).count('\r')+1])

In [ ]:
(fill.iloc[:,1])[4]

In [32]:
#dff.head(10)
fill = dff.iloc[:,0:3][~dff[0].str.contains("Код|БАКАЛАВРИАТ|Очная форма обучения|Очно-заочная форма обучения|СПЕЦИАЛИТЕТ|Заочная форма обучения", na=False)]

In [33]:
fill = fill[fill[2].str.contains("Образовательная|Совокупность", na=False)].reset_index()
fill

,index,0,1,2
0,5,37.03.01\rПсихология,очная,Образовательная программа:\rПсихология управления
1,8,37.03.01\rПсихология,очная,Образовательная программа:\rПсихология управления
2,11,38.03.01\rЭкономика,очная,Совокупность образовательных программ:\r1. Эко...
3,14,38.03.01\rЭкономика,очная,Совокупность образовательных программ:\r1. Эко...
4,17,38.03.04\rГосударственное и\rмуниципальное упр...,очная,Образовательная программа:\rАдминистративное у...
5,20,38.03.04\rГосударственное и\rмуниципальное упр...,очная,Образовательная программа:\rАдминистративное у...
6,23,40.03.01\rЮриспруденция,очная,Совокупность образовательных программ:\r1. Уго...
7,26,40.03.01\rЮриспруденция,очная,Совокупность образовательных программ:\r1. Уго...
8,30,42.03.05\rМедиакоммуникации,очная,Образовательная программа:\rМедиакоммуникации ...
9,38,37.03.01\rПсихология,очно-заочная,Образовательная программа:\rПсихология управления


In [ ]:
dff.head(10)
fil = dff.iloc[:,0:3][~dff[0].str.contains("Код|Очная форма обучения|Очно-заочная форма обучения|Заочная форма обучения", na=False)]

In [43]:
fil = fil[ fil[0].str.contains('0.|ФИЛИАЛ|БАКАЛАВРИАТ|СПЕЦИАЛИТЕТ', na=False) ].reset_index(drop = True)#.iloc[:, 1:3]
fil.head()
st = fil.iloc[:,0]
st

0                                       АЛТАЙСКИЙ ФИЛИАЛ
1                                            БАКАЛАВРИАТ
2                                   37.03.01\rПсихология
3                                   37.03.01\rПсихология
4                                    38.03.01\rЭкономика
5                                    38.03.01\rЭкономика
6      38.03.04\rГосударственное и\rмуниципальное упр...
7      38.03.04\rГосударственное и\rмуниципальное упр...
8                                40.03.01\rЮриспруденция
9                                40.03.01\rЮриспруденция
10                           42.03.05\rМедиакоммуникации
11                                  37.03.01\rПсихология
12                                   38.03.01\rЭкономика
13     38.03.04\rГосударственное и\rмуниципальное упр...
14                               40.03.01\rЮриспруденция
15                               40.03.01\rЮриспруденция
16                            40.03.01\rЮриспруденция***
17                           42

In [ ]:
result = re.search(r'Analy', 'AV Analytics Vidhya AV')
print(result[0])

In [ ]:
w = re.findall(r'\b[()]', df_one[1][14])
print(w)

In [ ]:
df_one[1][14].split('\r')[0]

In [ ]:
df_one[df_one[1].str.contains('\r') == df_one[1].str.contains('()')]

In [ ]:
df_one[df_one[1].str.contains('\(')]

In [ ]:
df_one[1][57].replace('\r', ' ').split('(')[0].strip()

In [ ]:
df_one[1][14]
#.replace('\r', ' ').replace('(ФММС ИУРР)', '')

In [ ]:
for i in range(0, len(df_one.iloc[:,0])):
    #if (df_one.iloc[i,0].count('\r') ==1) :
        print(df_one[1][i].replace('\r', ' ').split('(')[0].strip())
        print(df_one[1][i].replace('\r', ' ').split('(')[1].split(')')[0].strip())

In [ ]:
df_one.iloc[8,0].count('\r')
df_one[1][14].split('\r',2)

In [ ]:
df_one[1][14].split('\r',2)[0].split('(')[0].strip()
df_one[1][14].split('\r',2)[0].split('(')[1].strip(')')
df_one[1][14].split('\r',2)[1].split('(')[0].strip()
df_one[1][14].split('\r',2)[1].split('(')[1].strip(')')
df_one[1][14].split('\r',2)[2].split('(')[0].strip()
df_one[1][14].split('\r',2)[2].split('(')[1].strip(')')

In [ ]:
i = 0
while (i < df_one[1][14].count('\r')):
    print(df_one[1][14].split('\r',2)[i].split('(')[0].strip())
    print(df_one[1][14].split('\r',2)[i].split('(')[1].strip(')'))
    i = i + 1

In [ ]:
q=[]
w=[]

def mosc(strr):
    for j in range(0, len(strr)):
        i = 0
        while (i <= strr[j].count('\r')):
            if(strr[j].__contains__('\r)')):
                strr[j] = strr[j].replace('\r',' ')
            q.append(strr[j].split('\r',strr[j].count('\r'))[i].split('(')[0].strip())
        #print(strr.split('\r',strr.count('\r'))[i].split('(')[0].strip())
        #w.append(strr.split('\r',strr.count('\r'))[i].split('(')[1].strip(')'))
        #print(strr.split('\r',strr.count('\r'))[i].split('(')[1].strip(')'))
            i = i+1
mosc(df_one[1])
mos = pd.DataFrame()
mos['q'] = q
#mos['w'] = w
mos

In [ ]:
df_one[1][2]

In [ ]:
df_one[1]

In [ ]:
k = []
#op = df_one[1][10]
def qwe():
    for j in range(0, len(df_one[1])):
        op = df_one[1][j]
        
        k.append(t(op))
    #print(q)
qwe()

In [ ]:
mos = pd.DataFrame()
mos['q'] = df_one[1].map(lambda x: t(x))
#mos['w'] = w
mos

In [ ]:

def t(op):
    q = []
    w = []
    i = 0
    if (op.count('\r') == 0):
        q.append(op.split('\r',op.count('\r'))[i].split('(')[0].strip())
    while (op.count('\r') != 0):
                #if(op.__contains__('\r)')):
                    #op = op.replace('\r',' ')
            #q.append(strr[j].split('\r',strr[j].count('\r'))[i].split('(')[0].strip())
                #print()
                #q.append(op.split('\r',op.count('\r'))[i].split('(')[0].strip(''))
                q.append(''.join(op.split('\r',op.count('\r'))[i].split('(')[0].strip()))
                #w.append(op.split('\r',op.count('\r'))[0].split('(')[1].strip(')'))
        #w.append(strr.split('\r',strr.count('\r'))[i].split('(')[1].strip(')'))
        #print(strr.split('\r',strr.count('\r'))[i].split('(')[1].strip(')'))
                i = i+1
    return(q)
    #return(q)


In [ ]:
t(df_one[1][1])

In [ ]:
pop = df_one[1][1]
pop.split('\r',op.count('\r'))[1].split('(')[1].strip()
#pop

In [ ]:
print(pop.split('\r',op.count('\r'))[1].split('(')[1].strip())
#print(pop.split('\r',op.count('\r'))[2].split('(')[1].strip(')'))

In [49]:
filiall

,код,стандарт,программа,филиал,уровень
0,37.03.01,Психология,Психология управления,Алтайский филиал,Бакалавриат
1,38.03.01,Экономика,Экономика и управление финансами,Алтайский филиал,Бакалавриат
2,38.03.01,Экономика,Налоги и налогообложение,Алтайский филиал,Бакалавриат
3,38.03.04,Государственное и муниципальное управление,Административное управление,Алтайский филиал,Бакалавриат
4,40.03.01,Юриспруденция,Уголовно-правовой профиль,Алтайский филиал,Бакалавриат
5,40.03.01,Юриспруденция,Государственно-правовой профиль,Алтайский филиал,Бакалавриат
6,40.03.01,Юриспруденция,Гражданско-правовой профиль,Алтайский филиал,Бакалавриат
7,42.03.05,Медиакоммуникации,Медиакоммуникации и продвижение в цифровой среде,Алтайский филиал,Бакалавриат
8,38.03.04,Государственное и муниципальное управление,Управление в сфере здравоохранения,Алтайский филиал,Бакалавриат
9,40.05.04,Судебная и прокурорская деятельность,Прокурорская деятельность,Алтайский филиал,Специалитет


In [55]:
fill[fill.iloc[:,3].str.contains('Совокупность')][2].replace('Совокупность образовательных программ:\r1.', '')

2      Совокупность образовательных программ:\r1. Эко...
3      Совокупность образовательных программ:\r1. Эко...
6      Совокупность образовательных программ:\r1. Уго...
7      Совокупность образовательных программ:\r1. Уго...
11     Совокупность образовательных программ:\r1. Адм...
12     Совокупность образовательных программ:\r1. Уго...
13     Совокупность образовательных программ:\r1. Уго...
14     Совокупность образовательных программ:\r1. Уго...
28     Совокупность образовательных программ:\r1. Эко...
31     Совокупность образовательных программ:\r1. Эко...
37     Совокупность образовательных программ:\r1. Эко...
102    Совокупность образовательных программ:\r1. Гос...
103    Совокупность образовательных программ:\r1. Гос...
105    Совокупность образовательных программ:\r1. Гос...
164    Совокупность образовательных программ:\r1. Бух...
169    Совокупность образовательных программ:\r1. Бух...
173    Совокупность образовательных программ:\r1. Эфф...
174    Совокупность образовател

In [63]:
fill[fill.iloc[:,3].str.contains('Совокупность')][2].reset_index(drop = True)[20].replace('Совокупность образовательных программ:','')

'\r1. Организация таможенного контроля\r2. Внешнеторговая, транспортная и таможенная\rлогистика'

In [73]:
fill[fill.iloc[:,3].str.contains('Совокупность')][2].reset_index(drop = True)[20].count('.')

2

In [76]:
fill[fill.iloc[:,3].str.contains('Совокупность')][2].reset_index(drop = True)[20].split('.')

['Совокупность образовательных программ:\r1',
 ' Организация таможенного контроля\r2',
 ' Внешнеторговая, транспортная и таможенная\rлогистика']

In [ ]:
for j in range(1, fill[fill.iloc[:,3][i].count(.)):
        print(fill[fill.iloc[:,3][i].split('.')[j].replace('\r', ''))


In [ ]:
fill.iloc[:,3][i]

In [91]:
for j in range(1, fill[fill.iloc[:,3].str.contains('Совокупность')][2].reset_index(drop = True)[20].count('.')):
        print(fill[fill.iloc[:,3].str.contains('Совокупность')][2].reset_index(drop = True)[20].split('.')[j].replace('\r', ' '))

 Организация таможенного контроля 2


In [92]:
first = df[df[0].str.contains('СПЕЦИАЛИТЕТ|БАКАЛАВРИАТ|0.')][0].reset_index(drop = True)
second = df[~df[1].str.contains('Направленность', na = True)][1].reset_index(drop = True)

In [94]:
second[43]

'Государственное управление и правовое регулирование (с\rуглубленным изучением иностранных языков и правовых\rдисциплин) (ИГСУ)\rУправление территориальным развитием (многопрофильный\rбакалавриат) (ИГСУ)\rЦифровое государство (ИГСУ)\rЭкономика и управление (с углубленным изучением иностранного\rязыка и экономических дисциплин) (ИГСУ)'